<a href="https://colab.research.google.com/github/AnitaTasnim/WrongWayDrivingCode/blob/main/WrongWayDriving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [1]:
!git clone https://github.com/Usmankhujaev/Wrong-direction-drivers-detection.git
%cd Wrong-direction-drivers-detection


Cloning into 'Wrong-direction-drivers-detection'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 88 (delta 12), reused 4 (delta 4), pack-reused 73 (from 1)
Receiving objects: 100% (88/88), 1.53 MiB | 17.37 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/Wrong-direction-drivers-detection


In [2]:
!ls


detector_car_person.py	model_data	    requirenment.yml  train.py
KalmanFilter.py		object_tracking.py  result	      yolo3
kmeans.py		README.md	    tracker.py


In [3]:
!pip install -q \
opencv-python \
opencv-contrib-python \
pillow \
matplotlib \
filterpy \
numpy \
tensorflow==2.16.1 \
keras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.9/589.9 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 131.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 124.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which 

In [4]:
!wget https://pjreddie.com/media/files/yolov3.weights -O model_data/yolov3.weights


--2025-12-17 06:31:48--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 104.21.88.156, 172.67.185.199, 2606:4700:3030::ac43:b9c7, ...
Connecting to pjreddie.com (pjreddie.com)|104.21.88.156|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.pjreddie.com/files/yolov3.weights [following]
--2025-12-17 06:31:48--  https://data.pjreddie.com/files/yolov3.weights
Resolving data.pjreddie.com (data.pjreddie.com)... 172.67.185.199, 104.21.88.156, 2606:4700:3037::6815:589c, ...
Connecting to data.pjreddie.com (data.pjreddie.com)|172.67.185.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘model_data/yolov3.weights’

model_data/yolov3.w 100%[===================>] 236.52M  72.9MB/s    in 3.2s    

2025-12-17 06:31:52 (72.9 MB/s) - ‘model_data/yolov3.weights’ saved [248007048/248007048]



In [5]:
!ls model_data


car_class_2.txt  yolo_anchors.txt  yolov3.weights


In [6]:
!ls result


algorithm2.png
result.png
wrong_direction_20191118_181432_2_01.jpg
wrong_direction_20191118_181432_3_01.jpg
wrong_direction_20191118_181433_3_00.jpg
wrong_direction_20191118_181433_3_01.jpg
wrong_direction_20191118_181501_2_01.jpg
wrong_direction_20191118_181502_2_01.jpg
wrong_direction_20191118_181502_3_00.jpg
wrong_direction_20191118_181503_3_00.jpg


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#kalman_filter.py

In [ ]:


# Import python libraries
import numpy as np
from KalmanFilter import KalmanFilterClass

from scipy.optimize import linear_sum_assignment


class Track(object):


    def __init__(self, prediction, trackIdCount):

        self.track_id = trackIdCount  # identification of each track object
        self.KF = KalmanFilterClass()  # KF instance to track this object
        self.prediction = np.asarray(prediction)  # predicted centroids (x,y)
        self.skipped_frames = 0  # number of frames skipped undetected
        self.trace = []  # trace path


class Tracker(object):


    def __init__(self, dist_thresh, max_frames_to_skip, max_trace_length,
                 trackIdCount):

        self.dist_thresh = dist_thresh
        self.max_frames_to_skip = max_frames_to_skip
        self.max_trace_length = max_trace_length
        self.tracks = []
        self.trackIdCount = trackIdCount

    def Update(self, detections):

        # Create tracks if no tracks vector found
        if (len(self.tracks) == 0):
            for i in range(len(detections)):
                track = Track(detections[i], self.trackIdCount)
                self.trackIdCount += 1
                self.tracks.append(track)

        # Calculate cost using sum of square distance between
        # predicted vs detected centroids
        N = len(self.tracks)
        M = len(detections)
        cost = np.zeros(shape=(N, M))   # Cost matrix
        for i in range(len(self.tracks)):
            for j in range(len(detections)):
                try:
                    diff = self.tracks[i].prediction - detections[j]
                    distance = np.sqrt(diff[0][0]*diff[0][0] +
                                       diff[1][0]*diff[1][0])
                    cost[i][j] = distance
                except:
                    pass

        # Let's average the squared ERROR
        cost = (0.5) * cost
        # Using Hungarian Algorithm assign the correct detected measurements
        # to predicted tracks
        assignment = []
        for _ in range(N):
            assignment.append(-1)
        row_ind, col_ind = linear_sum_assignment(cost)
        for i in range(len(row_ind)):
            assignment[row_ind[i]] = col_ind[i]

        # Identify tracks with no assignment, if any
        un_assigned_tracks = []
        for i in range(len(assignment)):
            if (assignment[i] != -1):
                # check for cost distance threshold.
                # If cost is very high then un_assign (delete) the track
                if (cost[i][assignment[i]] > self.dist_thresh):
                    assignment[i] = -1
                    un_assigned_tracks.append(i)
                pass
            else:
                self.tracks[i].skipped_frames += 1

        # If tracks are not detected for long time, remove them
        del_tracks = []
        for i in range(len(self.tracks)):
            if (self.tracks[i].skipped_frames > self.max_frames_to_skip):
                del_tracks.append(i)
        if len(del_tracks) > 0:  # only when skipped frame exceeds max
            for id in del_tracks:
                if id < len(self.tracks):
                    del self.tracks[id]
                    del assignment[id]
                else:
                    print("ERROR: id is greater than length of tracks")

        # Now look for un_assigned detects
        un_assigned_detects = []
        for i in range(len(detections)):
                if i not in assignment:
                    un_assigned_detects.append(i)

        # Start new tracks
        if(len(un_assigned_detects) != 0):
            for i in range(len(un_assigned_detects)):
                track = Track(detections[un_assigned_detects[i]],
                              self.trackIdCount)
                self.trackIdCount += 1
                self.tracks.append(track)

        # Update KalmanFilter state, lastResults and tracks trace
        for i in range(len(assignment)):
            self.tracks[i].KF.predict()

            if(assignment[i] != -1):
                self.tracks[i].skipped_frames = 0
                self.tracks[i].prediction = self.tracks[i].KF.correct(
                                            detections[assignment[i]], 1)
            else:
                self.tracks[i].prediction = self.tracks[i].KF.correct(
                                            np.array([[0], [0]]), 0)

            if(len(self.tracks[i].trace) > self.max_trace_length):
                for j in range(len(self.tracks[i].trace) -
                               self.max_trace_length):
                    del self.tracks[i].trace[j]

            self.tracks[i].trace.append(self.tracks[i].prediction)
            self.tracks[i].KF.lastResult = self.tracks[i].prediction



#tracker

In [10]:


# Import python libraries
import numpy as np
from KalmanFilter import KalmanFilterClass

from scipy.optimize import linear_sum_assignment


class Track(object):


    def __init__(self, prediction, trackIdCount):

        self.track_id = trackIdCount  # identification of each track object
        self.KF = KalmanFilterClass()  # KF instance to track this object
        self.prediction = np.asarray(prediction)  # predicted centroids (x,y)
        self.skipped_frames = 0  # number of frames skipped undetected
        self.trace = []  # trace path


class Tracker(object):


    def __init__(self, dist_thresh, max_frames_to_skip, max_trace_length,
                 trackIdCount):

        self.dist_thresh = dist_thresh
        self.max_frames_to_skip = max_frames_to_skip
        self.max_trace_length = max_trace_length
        self.tracks = []
        self.trackIdCount = trackIdCount

    def Update(self, detections):

        # Create tracks if no tracks vector found
        if (len(self.tracks) == 0):
            for i in range(len(detections)):
                track = Track(detections[i], self.trackIdCount)
                self.trackIdCount += 1
                self.tracks.append(track)

        # Calculate cost using sum of square distance between
        # predicted vs detected centroids
        N = len(self.tracks)
        M = len(detections)
        cost = np.zeros(shape=(N, M))   # Cost matrix
        for i in range(len(self.tracks)):
            for j in range(len(detections)):
                try:
                    diff = self.tracks[i].prediction - detections[j]
                    distance = np.sqrt(diff[0][0]*diff[0][0] +
                                       diff[1][0]*diff[1][0])
                    cost[i][j] = distance
                except:
                    pass

        # Let's average the squared ERROR
        cost = (0.5) * cost
        # Using Hungarian Algorithm assign the correct detected measurements
        # to predicted tracks
        assignment = []
        for _ in range(N):
            assignment.append(-1)
        row_ind, col_ind = linear_sum_assignment(cost)
        for i in range(len(row_ind)):
            assignment[row_ind[i]] = col_ind[i]

        # Identify tracks with no assignment, if any
        un_assigned_tracks = []
        for i in range(len(assignment)):
            if (assignment[i] != -1):
                # check for cost distance threshold.
                # If cost is very high then un_assign (delete) the track
                if (cost[i][assignment[i]] > self.dist_thresh):
                    assignment[i] = -1
                    un_assigned_tracks.append(i)
                pass
            else:
                self.tracks[i].skipped_frames += 1

        # If tracks are not detected for long time, remove them
        del_tracks = []
        for i in range(len(self.tracks)):
            if (self.tracks[i].skipped_frames > self.max_frames_to_skip):
                del_tracks.append(i)
        if len(del_tracks) > 0:  # only when skipped frame exceeds max
            for id in del_tracks:
                if id < len(self.tracks):
                    del self.tracks[id]
                    del assignment[id]
                else:
                    print("ERROR: id is greater than length of tracks")

        # Now look for un_assigned detects
        un_assigned_detects = []
        for i in range(len(detections)):
                if i not in assignment:
                    un_assigned_detects.append(i)

        # Start new tracks
        if(len(un_assigned_detects) != 0):
            for i in range(len(un_assigned_detects)):
                track = Track(detections[un_assigned_detects[i]],
                              self.trackIdCount)
                self.trackIdCount += 1
                self.tracks.append(track)

        # Update KalmanFilter state, lastResults and tracks trace
        for i in range(len(assignment)):
            self.tracks[i].KF.predict()

            if(assignment[i] != -1):
                self.tracks[i].skipped_frames = 0
                self.tracks[i].prediction = self.tracks[i].KF.correct(
                                            detections[assignment[i]], 1)
            else:
                self.tracks[i].prediction = self.tracks[i].KF.correct(
                                            np.array([[0], [0]]), 0)

            if(len(self.tracks[i].trace) > self.max_trace_length):
                for j in range(len(self.tracks[i].trace) -
                               self.max_trace_length):
                    del self.tracks[i].trace[j]

            self.tracks[i].trace.append(self.tracks[i].prediction)
            self.tracks[i].KF.lastResult = self.tracks[i].prediction



ModuleNotFoundError: No module named 'numpy.char'

#detector_car_person.py

In [9]:
# -*- coding: utf-8 -*-
"""
Class definition of YOLO_v3 style detection model on image and video
"""

import colorsys
import os
from timeit import default_timer as timer
import datetime as dt
import cv2
from tracker import Track, Tracker
import random as rand
import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw
from collections import Counter
from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
from keras.utils import multi_gpu_model

class YOLO(object):
    _defaults = {
        "model_path": 'model/trained_weights_final_1749_2.h5',
        "anchors_path": 'model_data/yolo_anchors.txt',
        "classes_path": 'model_data/car_class_2.txt',
        "score" : 0.3,
        "iou" : 0.40,
        "model_image_size" : (416, 416),
        "gpu_num" : 1,
    }
    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')
        h, w = image.size
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })
        centroids = []
        all_classes =[]
        lane_number = 0
        predicted_class = ""

        for i, c in reversed(list(enumerate(out_classes))):

            predicted_class = self.class_names[c]
            all_classes.append(predicted_class)
            box = out_boxes[i]
            top, left, bottom, right = box
            #y1, x1, y2, x2
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(w, np.floor(bottom + 0.5).astype('int32'))
            right = min(h, np.floor(right + 0.5).astype('int32'))

            mid_x = int((left+right)/2)
            mid_y = int((bottom+top)/2)
            image = np.array(image)
            h, w, d= image.shape
            centroids.append(np.round(np.array([[mid_y], [mid_x]])))


            #-------draw rectangles if car is detected--------------
            # uncomment these lines to the see the boxes
            #if c == 0:
                #cv2.rectangle(image, (left, top), (right, bottom), (255,255,0), 1)
                #cv2.putText(image, "Car", (int(left), int(top)), cv2.FONT_HERSHEY_SIMPLEX, 0.4,(255,255,0),1)

            if c == 1:
                cv2.rectangle(image, (left, top), (right, bottom), (0,0,255), 1)
                cv2.putText(image, "Person", (int(left), int(top)), cv2.FONT_HERSHEY_SIMPLEX, 0.4,(0,0,255),1)
                if (mid_y > h/3.86  and mid_y<=h/3.26):
                    lane_number = 1
                if (mid_y > h/2.42  and mid_y <=h/1.41):
                    lane_number = 2
                if (mid_y > h/1.41 and mid_y <=h):
                    lane_number = 3

                now = dt.datetime.now()
                raw_dir = './result/'+predicted_class+"_"
                lane_ = str(lane_number)+'_{0:02d}'.format(i)
                seconds =  now.second
                saving_name = "%s%s%s.jpg"%(raw_dir, now.strftime("%Y%m%d_%H%M%S_"), lane_)
                if os.path.exists(saving_name) == False and seconds%5 == 0:
                    if os.path.exists("./result") == False:
                        os.mkdir("./result")
                    print(saving_name)
                    resized = resize(image)
                    cv2.imwrite(saving_name,resized)

            try:
                image = Image.fromarray(image)
            except:
                print('no cars')

        return image, centroids, out_scores, all_classes, out_boxes

    def close_session(self):
        self.sess.close()
def resize(img, scale=200):
    # percent of original size
    width = int(img.shape[1] * scale / 100)
    height = int(img.shape[0] * scale / 100)
    dim = (width, height)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized

def detect_video(yolo, video_path, output_path=""):
    vid = cv2.VideoCapture(video_path)
    tracker = Tracker(30,0,6,0)
    width = 400
    height = 300

    skip_frame_count = 0
    track_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
                    (0, 255, 255), (255, 0, 255), (255, 127, 255),
                    (127, 0, 255), (127, 0, 127), (50,50,98),(37,37,47)]
    pause = False
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    video_FourCC = int(vid.get(cv2.CAP_PROP_FOURCC))
    video_fps = vid.get(cv2.CAP_PROP_FPS)
    video_size = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    isOutput = True if output_path != "" else False
    if isOutput:
        print("!!! TYPE:", type(output_path), type(video_FourCC), type(video_fps), type(video_size))
        out = cv2.VideoWriter(output_path, video_FourCC, video_fps,(int(0.78*width)-int(0.109*width), height-int(height*0.0608)))

    fps = "FPS: ??"
    accum_time = 0
    curr_fps = 0
    clr = 0

    frame_counter = 0
    direction=""
    saving_name=""
    (dX, dY) = (0, 0)

    lane_number = 0
    centers = []

    prev_time = timer()
    while True:
        return_value, frame = vid.read()

        if return_value is False:
            print('frame is empty: break')
            break

        orig_frame = frame.copy()
        frame = cv2.resize(frame, (width, height))
        frame = frame[int(0.0608*height):height, int(0.109*width):int(0.78*width)]
        h,w,d=frame.shape

        frame_counter+=1

        image = Image.fromarray(frame)
        new_result, centers, scores, out_classes, bbox = yolo.detect_image(image)


        result = np.array(new_result)
        curr_time = timer()
        exec_time = curr_time - prev_time
        prev_time = curr_time
        accum_time = accum_time + exec_time
        curr_fps = curr_fps + 1

        if len(centers)>0 and len(out_classes)>0:
            tracker.Update(centers)
            for i in range(len(tracker.tracks)):
                x1=0
                y1=0
                x2=0
                y2=0
                if(len(tracker.tracks[i].trace)>1):
                    for j in range(len(tracker.tracks[i].trace)-1):
                        y1 = tracker.tracks[i].trace[j][0][0]
                        x1 = tracker.tracks[i].trace[j][1][0]
                        y2 = tracker.tracks[i].trace[j+1][0][0]
                        x2 = tracker.tracks[i].trace[j+1][1][0]
                        clr = tracker.tracks[i].track_id
                        try:
                            some_position = tracker.tracks[i].trace[2][1][0]
                            some_position_y = tracker.tracks[i].trace[2][0][0]

                        except:
                            some_position = tracker.tracks[i].trace[1][1][0]
                            some_position_y = tracker.tracks[i].trace[1][0][0]

                        #cv2.circle(result, (int(x2), int(y2)), 2, track_colors[clr%3], -1)
                        cv2.arrowedLine(result, (int(some_position), int(some_position_y)), (int(x2), int(y2)),track_colors[clr%3], line_type=cv2.LINE_AA, thickness=1)

                        dX = x2-some_position
                        #dXCheck = x2-x1
                        dY = y2-y1
                    (dirX, dirY) = ("", "")
                    #print(dX)
                    if np.abs(dX)>=12:
                        dirX = "East" if np.sign(dX) == 1 else "West"
                    #if np.abs(dY)>=4:
                    #    dirY = "North" if np.sign(dY) == 1 else "South"

                    if dirX != "" and dirY != "":
                        direction = "{}-{}".format(dirY, dirX)
                    else:
                        direction = dirX if dirX != "" else dirY

                    if direction == "East":
                        if (y2 > 0 and y2<=h/2.28):
                            lane_number = 1
                        if (y2 > h/3.68  and y2 <=h/1.35):
                            lane_number = 2
                        if (y2 > h/1.35 and y2 <=h):
                            lane_number = 3

                        cv2.putText(result, 'WRONG DIRECTION', (int(x1)-25, int(y1)-25), cv2.FONT_HERSHEY_SIMPLEX, 0.30, (0, 0, 255), 1)
                        now = dt.datetime.now()
                        raw_dir = './result/wrong_direction_'
                        lane_ = str(lane_number)+'_{0:02d}'.format(i)
                        seconds =  now.second
                        right_now =  now.strftime("%Y%m%d_%H%M%S_")
                        saving_name = "%s%s%s.jpg"%(raw_dir,right_now,lane_)

                        if os.path.exists(saving_name) == False:
                            if os.path.exists("./result") == False:
                                os.mkdir("./result")
                            print(saving_name)
                            resized = resize(result)
                            cv2.imwrite(saving_name,resized)

                    else:
                        cv2.putText(result, direction, (int(x1)-25, int(y1)-25), cv2.FONT_HERSHEY_SIMPLEX, 0.30, (0, 255, 0), 1)


        if accum_time > 1:
            accum_time = accum_time - 1
            fps = "FPS: " + str(curr_fps)
            curr_fps = 0

        cv2.putText(result, "Count: "+str(len(centers)), (3, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 1 )
        cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.40, color=(0, 255, 0), thickness=1)

        cv2.namedWindow("result", cv2.WINDOW_NORMAL)
        cv2.imshow("result", result)
        k = cv2.waitKey(1) & 0xff
        if isOutput:
            try:
                out.write(result)
            except: continue
        if k == ord('q'):
            break
        if k == 112:  # 'p' has been pressed. this will pause/resume the code.
            pause = not pause
            if (pause is True):
                print("Code is paused. Press 'p' to resume..")
                while (pause is True):
                    # stay in this loop until
                    key = cv2.waitKey(30) & 0xff
                    if key == 112:
                        pause = False
                        print("Resume code..!!")
                        break
    yolo.close_session()

ModuleNotFoundError: No module named 'numpy.char'

#object_tracking.py

In [8]:
import sys
import cv2
import argparse
from detector_car_person import YOLO, detect_video
from PIL import Image

def detect_img(yolo):
    while True:
        img = input('Input image filename:')
        try:
            image = Image.open(img)
        except:
            print('Open Error! Try again!')
            continue
        else:
            r_image = yolo.detect_image(image)
            #r_image.show()
    yolo.close_session()

FLAGS = None

if __name__ == '__main__':
    # class YOLO defines the default value, so suppress any default here
    parser = argparse.ArgumentParser(argument_default=argparse.SUPPRESS)
    '''
    Command line options
    '''
    parser.add_argument(
        '--model', type=str,
        help='path to model weight file, default ' + YOLO.get_defaults("model_path")
    )

    parser.add_argument(
        '--anchors', type=str,
        help='path to anchor definitions, default ' + YOLO.get_defaults("anchors_path")
    )

    parser.add_argument(
        '--classes', type=str,
        help='path to class definitions, default ' + YOLO.get_defaults("classes_path")
    )

    parser.add_argument(
        '--gpu_num', type=int,
        help='Number of GPU to use, default ' + str(YOLO.get_defaults("gpu_num"))
    )

    parser.add_argument(
        '--image', default=False, action="store_true",
        help='Image detection mode, will ignore all positional arguments'
    )
    '''
    Command line positional arguments -- for video detection mode
    '''
    parser.add_argument(
        "--input", nargs='?', type=str,required=False,default='./path2your_video',
        help = "Video input path"
    )

    parser.add_argument(
        "--output", nargs='?', type=str, default="",
        help = "[Optional] Video output path"
    )

    FLAGS = parser.parse_args()

    if FLAGS.image:
        """
        Image detection mode, disregard any remaining command line arguments
        """
        print("Image detection mode")
        if "input" in FLAGS:
            print(" Ignoring remaining command line arguments: " + FLAGS.input + "," + FLAGS.output)
        #This function image path.
        detect_img(YOLO(**vars(FLAGS)))
    elif "input" in FLAGS:
        detect_video(YOLO(**vars(FLAGS)), FLAGS.input, FLAGS.output)
    else:
        print("Must specify at least video_input_path.  See usage with --help.")

ModuleNotFoundError: No module named 'numpy.char'

#next

In [ ]:
!python object_tracking.py \
  --input /content/drive/MyDrive/Jakir_Sir_office/WWD/test_videos/car.mp4


2025-12-13 20:20:02.867412: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-13 20:20:03.000656: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-13 20:20:08.176939: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
Traceback (most recent call last):
  File "/content/Wrong-direction-drivers-detection/object_tracking.py", line 4, in <module>
    from detector_car_person import YOLO, detect_video
  File "/content/Wrong-direction-drivers-detection/detector_car_person.py", line 19, in <module>
    from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
  File "/content/Wrong-direction-drivers-detecti

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
!{sys.executable} -m pip install -q opencv-python pillow matplotlib filterpy tensorflow==2.16.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement tensorflow==2.12 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.12


In [ ]:
# This cell was a duplicate of a previous git clone command and is now empty.